In [1]:
import elasticsearch
import unicodedata
import csv
from elasticsearch_dsl.aggs import Terms, Nested
from elasticsearch_dsl.search import Search
from elasticsearch_dsl import Q
import networkx as nx
import pandas as pd
from pandas.io.json import json_normalize
import json
import pickle
import datetime
import numpy as np

es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])

#####################################################################
# Make your Query with elasticsearch_dsl
#####################################################################
res = Search(using=es, index="filebeat*").filter('range',**{"@timestamp":{'gte': '2018-10-12T00:00:00.000Z', 'lt' : '2019-03-29T23:59:59.999Z'}}).params(request_timeout=10000)\
.query("regexp", message="[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}")\
.exclude("match", message='bound')\
.exclude('bool', filter=Q('match', message='ci')& Q('match', message='info'))\
.exclude("match", message='DataSourceOpenStack.py')\
.exclude("match", message='dhclient')\
.exclude("match", message='dhcp')\
.exclude("match", message='dhcp.py')\
.exclude("match", message='DHCPACK')\
.exclude("match", message='DHCPDISCOVER')\
.exclude("match", message='DHCPOFFER')\
.exclude("match", message='DHCPREQUEST')\
.exclude("match", message='ephemeral')\
.exclude("match", message='filebeat')\
.exclude("match", message='kibana')\
.exclude("match", message='launch_jupyter.sh')\
.exclude("match", message='ntpd')\
.exclude("match", message='packetbeat')\
.exclude('bool', filter=Q('match', message='Resolving')& Q('match', message='URL')& Q('match', message='http'))\
.exclude("match", message='UFW')\
.exclude("match", message='url_helper.py')\
.exclude("match", message='dhcpd')\
.exclude("match", message='openstack.py')\
.exclude("match", message='util.py')\
.query("match", message='Accepted')

#####################################################################
# Print the underlying JSON representation of your Query
#####################################################################
#res_json = res.to_dict()
#print(res_json)
    
#####################################################################
# Response execution
#####################################################################
response = res.execute()
#####################################################################

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

Filtered logs: 16995
Total number of logs: 16995


In [3]:
## # Normalize each response (flow in response) and append to dataframe
#####################################################################
print("#####################")
print(datetime.datetime.now())
print("#####################")
counter = 0
logs_list =[]

for hits in res.scan():
    temp_json = hits.to_dict()
    temp_json['meta_id'] = hits.meta.id
    temp_json['meta_index'] = hits.meta.index
    logs_list.append(temp_json)
    counter+=1
    if (counter % 500000) == 0:
        print(datetime.datetime.now())
        print(counter)
        #print("\n")
print(datetime.datetime.now())
print(counter)
with open('/Users/ejl334/IDS_OS_log_tagging_and_LSTM/os_logs.json','w') as outfile:
    json.dump(logs_list, outfile, ensure_ascii=False, indent=2)
    
#####################################################################
# Close File
#####################################################################
#main_pandas_df.head(50)
outfile.close()

#####################
2019-10-21 15:03:56.435996
#####################
2019-10-21 15:04:07.594547
16995


# Load JSON File

In [6]:
import json

with open('/Users/ejl334/IDS_OS_log_tagging_and_LSTM/os_logs.json') as json_file:  
    data = json.load(json_file)

json_file.close()
test_data = data[0]
print(json.dumps(test_data, indent=4, sort_keys=True))

{
    "@timestamp": "2018-12-09T00:30:25.000Z",
    "@version": "1",
    "PID": "3403",
    "accesedUser": "root",
    "beat": {
        "hostname": "elk-master",
        "name": "elk-master",
        "version": "6.4.2"
    },
    "geoip": {
        "city_name": "San Antonio",
        "continent_code": "NA",
        "country_code2": "US",
        "country_code3": "US",
        "country_name": "United States",
        "dma_code": 641,
        "ip": "66.69.79.125",
        "latitude": 29.6255,
        "location": {
            "lat": 29.6255,
            "lon": -98.6299
        },
        "longitude": -98.6299,
        "postal_code": "78256",
        "region_code": "TX",
        "region_name": "Texas",
        "timezone": "America/Chicago"
    },
    "host": {
        "name": "elk-master"
    },
    "input": {
        "type": "log"
    },
    "keyType": "RSA",
    "message": "Dec  8 14:30:25 elk-master sshd[3403]: Accepted publickey for root from 66.69.79.125 port 62735 ssh2: RSA SHA256:

# Load data into a Pandas Dataframe

In [11]:
import pandas as pd

df = pd.DataFrame.from_records(data)
df.head()
del data

In [12]:
#df['message']

In [ ]:
from pygrok import Grok

In [ ]:
patterns = []
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}avahi-autoipd.*Callout.*,%{SPACE}address%{SPACE}%{IP:src_ip_address}.*')# 0
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}avahi-autoipd.*Starting.*%{SPACE}address%{SPACE}%{IP:src_ip_address}.*')# 1
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}avahi-autoipd.*Successfully.*%{SPACE}address%{SPACE}%{IP:src_ip_address}.*')# 2
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Accepted%{SPACE}password%{SPACE}for%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 3
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Accepted%{SPACE}password%{SPACE}for%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 4
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Accepted%{SPACE}publickey%{SPACE}for%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 5
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Accepted%{SPACE}publickey%{SPACE}for%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 6
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Connection%{SPACE}closed%{SPACE}by%{SPACE}%{IP:src_ip_address}.*')# 7
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Connection%{SPACE}reset%{SPACE}by%{SPACE}%{IP:src_ip_address}.*')# 8
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Disconnected%{SPACE}from%{SPACE}%{IPORHOST:src_ip_address}.*')# 9
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*error:%{SPACE}maximum%{SPACE}authentication%{SPACE}attempts%{SPACE}exceeded%{SPACE}for%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 10
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*error:%{SPACE}maximum%{SPACE}authentication%{SPACE}attempts%{SPACE}exceeded%{SPACE}for%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 11
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Failed%{SPACE}none%{SPACE}for%{SPACE}invalid%{SPACE}user%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 12
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Failed%{SPACE}none%{SPACE}for%{SPACE}invalid%{SPACE}user%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 13
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Failed%{SPACE}password%{SPACE}for%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 14
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Failed%{SPACE}password%{SPACE}for%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 15
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*fatal:%{SPACE}packet_write_wait:%{SPACE}Connection%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 16
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}input_userauth_request: invalid%{SPACE}user%{SPACE}%{IP:src_ip_address}.*')# 17 
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}PAM.*authentication%{SPACE}failure.*%{IP:src_ip_address}%{SPACE}%{USER:local_user}.*')# 18
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}PAM.*authentication%{SPACE}failure.*%{IP:src_ip_address}%{SPACE}.*')# 19
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}pam_unix\(sshd:auth\).*authentication%{SPACE}failure.*%{IP:src_ip_address}%{SPACE}%{USER:local_user}.*')# 20
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}pam_unix\(sshd:auth\).*authentication%{SPACE}failure.*%{IP:src_ip_address}%{SPACE}.*')# 21
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}Received%{SPACE}disconnect%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 22
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*fatal:%{SPACE}ssh_dispatch_run_fatal:%{SPACE}Connection%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 23
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*fatal:%{SPACE}Unable%{SPACE}to%{SPACE}negotiate%{SPACE}with%{SPACE}%{IP:src_ip_address}.*') # 24

In [ ]:
def process(line, annotated_ips_l1):
    for j, ptrn in enumerate(patterns):
        gork_match = Grok(ptrn).match(line)
        if (gork_match == None):
            pass
        else:
            print("Log Message")
            print(line)
            print("Pattern")
            print(ptrn,"\n")
            print("Extracted features")
            print(gork_match,"\n")
            print("*************************************")

annotated_ips_l1=[]

In [10]:
for index, row in df.iterrows():
    print(row['message'])

Dec  8 14:30:25 elk-master sshd[3403]: Accepted publickey for root from 66.69.79.125 port 62735 ssh2: RSA SHA256:CX00z2IN5rD+Bg7CJChKzgrKMxDWLXrwd0G1EzBh/14
Oct 24 13:17:31 bigdata-vm-49 sshd[24734]: Accepted password for student from 10.1.1.65 port 37106 ssh2
Nov 25 12:57:01 elk-master sshd[12849]: Accepted publickey for root from 66.69.79.125 port 52452 ssh2: RSA SHA256:CX00z2IN5rD+Bg7CJChKzgrKMxDWLXrwd0G1EzBh/14
Oct 22 15:14:20 bigdata-vm-7 sshd[1777]: Accepted publickey for student from 10.1.1.54 port 50264 ssh2: RSA SHA256:I/+BglDZzJkO6Nw5RfVx5x+O7CxPMZ7MTEEPFtoX9oo
Oct 24 13:23:04 bigdata-vm-49 sshd[24950]: Accepted publickey for student from 10.1.1.54 port 41878 ssh2: RSA SHA256:I/+BglDZzJkO6Nw5RfVx5x+O7CxPMZ7MTEEPFtoX9oo
Oct 22 15:15:01 bigdata-vm-7 sshd[1827]: Accepted publickey for student from 10.1.1.54 port 50382 ssh2: RSA SHA256:I/+BglDZzJkO6Nw5RfVx5x+O7CxPMZ7MTEEPFtoX9oo
Nov 27 15:20:26 elk-master sshd[958]: Accepted publickey for root from 129.115.195.49 port 53449 ssh2: